In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import median
from sklearn.linear_model import LogisticRegression as LR
import sklearn as sk
from sklearn.linear_model import SGDClassifier as classifier

In [11]:
data= pd.read_csv(r"/Users/namanparuthi/Downloads/smoking_driking_dataset_Ver01.csv")

In [12]:
data.describe()

,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,DBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
count,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,...,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000,991346.000000
mean,47.614491,162.240625,63.284050,81.233358,0.980834,0.978429,1.031495,1.030476,122.432498,76.052627,...,56.936800,113.037692,132.141751,14.229824,1.094224,0.860467,25.989308,25.755051,37.136347,1.608122
std,14.181339,9.282957,12.514241,11.850323,0.605949,0.604774,0.174650,0.171892,14.543148,9.889365,...,17.238479,35.842812,102.196985,1.584929,0.437724,0.480530,23.493386,26.308599,50.424153,0.818507
min,20.000000,130.000000,25.000000,8.000000,0.100000,0.100000,1.000000,1.000000,67.000000,32.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.100000,1.000000,1.000000,1.000000,1.000000
25%,35.000000,155.000000,55.000000,74.100000,0.700000,0.700000,1.000000,1.000000,112.000000,70.000000,...,46.000000,89.000000,73.000000,13.200000,1.000000,0.700000,19.000000,15.000000,16.000000,1.000000
50%,45.000000,160.000000,60.000000,81.000000,1.000000,1.000000,1.000000,1.000000,120.000000,76.000000,...,55.000000,111.000000,106.000000,14.300000,1.000000,0.800000,23.000000,20.000000,23.000000,1.000000
75%,60.000000,170.000000,70.000000,87.800000,1.200000,1.200000,1.000000,1.000000,131.000000,82.000000,...,66.000000,135.000000,159.000000,15.400000,1.000000,1.000000,28.000000,29.000000,39.000000,2.000000
max,85.000000,190.000000,140.000000,999.000000,9.900000,9.900000,2.000000,2.000000,273.000000,185.000000,...,8110.000000,5119.000000,9490.000000,25.000000,6.000000,98.000000,9999.000000,7210.000000,999.000000,3.000000


In [13]:
data['SMK_stat_type_cd'].value_counts()

SMK_stat_type_cd
1    602441
3    213954
2    174951
Name: count, dtype: int64

In [14]:
data.drop(['sex','DRK_YN'], inplace=True, axis=1)


In [15]:
data.groupby('SMK_stat_type_cd').mean()

,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,DBP,...,tot_chole,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP
SMK_stat_type_cd,,,,,,,,,,,,,,,,,,,,,
1,48.455401,158.572582,59.306272,79.001125,0.949829,0.946845,1.032956,1.032227,121.177911,74.916407,...,195.336906,59.258090,113.364130,115.055730,13.638717,1.087614,0.803322,24.680925,22.813524,27.426485
2,50.112632,167.382296,69.406891,85.312877,1.013957,1.013486,1.038639,1.036216,125.345011,78.059588,...,194.951243,53.824900,112.836686,147.418752,14.967969,1.108768,0.965003,28.196804,29.959320,47.043515
3,43.203960,168.364555,69.477808,84.182930,1.041053,1.038697,1.021537,1.020850,123.583527,77.610842,...,196.672154,52.945241,112.282888,167.759654,15.290647,1.100947,0.935891,27.868313,30.599811,56.375763


Here by analysis of data for smokers and non-smokers, we can conclude that smoking affects triglyceride, gamma_GTP, SGOT_AST, SGOT_ALT, serum_creatinine levels heavily.

Model 1-Logistic Regression

In [16]:
y= data.loc[:,"SMK_stat_type_cd"]
x=data.loc[:, ('triglyceride', 'gamma_GTP', 'SGOT_AST','SGOT_ALT','serum_creatinine')]

In [17]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0,shuffle=True) 

In [18]:
model=LR(solver='newton-cg',multi_class='multinomial')
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


In [19]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [20]:
accuracy=sk.metrics.accuracy_score(y_test, y_pred)
accuracy*100

62.63529530438291

Model 2- skm regression

In [21]:
y= data.loc[:,"SMK_stat_type_cd"]
x=data.loc[:, ('triglyceride', 'gamma_GTP', 'SGOT_AST','SGOT_ALT','serum_creatinine')]

In [22]:
#normalising X 

import scipy
X=scipy.stats.zscore(x)

In [24]:
print(X)

        triglyceride  gamma_GTP  SGOT_AST  SGOT_ALT  serum_creatinine
0          -0.392788   0.056791 -0.212371  0.351404          0.290374
1          -0.109022  -0.201022 -0.254936  0.389415          0.082270
2          -0.275368   0.612081  0.894324  0.237373          0.082270
3          -0.255798  -0.379508  0.128151  0.313394          0.498477
4          -0.275368  -0.240685 -0.297501 -0.522835         -0.125833
...              ...        ...       ...       ...               ...
991341     -0.001387  -0.201022  0.000455  0.389415          0.290374
991342     -0.852685  -0.439003 -0.510327 -0.332783          0.498477
991343      0.243239  -0.399340  0.170716  0.047321         -0.125833
991344     -0.774404  -0.399340 -0.212371 -0.446814         -0.125833
991345      0.301949  -0.022536 -0.084675  0.655487          0.082270

[991346 rows x 5 columns]


In [25]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,shuffle=True)

In [26]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((743509, 5), (743509,), (247837, 5), (247837,))

In [27]:
classifier=classifier()
classifier.fit(X_train,y_train)
y_pred=classifier.predict(X_test)

In [28]:
y_pred

array([3, 1, 1, ..., 1, 1, 1])

In [29]:
accuracy=sk.metrics.accuracy_score(y_test, y_pred)
accuracy*100

61.893099093355715